In [1]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.2/387.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 40.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.2/201.2 kB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 117.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB

In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.datasets import ColumnCorpus
from flair.datasets import SentenceDataset
from flair.data import Token
from tqdm import tqdm
import gensim
from gensim.models.word2vec import Word2Vec
import pandas as pd
pd.set_option('max.colwidth', 0)

# Prepare Dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp /content/drive/MyDrive/mypos-ver.3.0-flair.txt .
!cp /content/drive/MyDrive/word2vec.model .

In [5]:
dataset_file = "mypos-ver.3.0-flair.txt"

# Define the columns in dataset
columns = {0: 'text', 1: 'pos'}

# Initialize the corpus
corpus = ColumnCorpus(data_folder='.', column_format=columns, train_file=dataset_file)

2023-11-16 09:19:56,059 Reading data from .
2023-11-16 09:19:56,067 Train: mypos-ver.3.0-flair.txt
2023-11-16 09:19:56,068 Dev: None
2023-11-16 09:19:56,070 Test: None
2023-11-16 09:20:11,239 No test split found. Using 0% (i.e. 4320 samples) of the train split as test data
2023-11-16 09:20:11,267 No dev split found. Using 0% (i.e. 3888 samples) of the train split as dev data


# Custom Word Embeddings (word2vec)

In [6]:
from flair.embeddings import TokenEmbeddings
from gensim.models import Word2Vec
from flair.data import Sentence
from typing import List, Dict, Any
import numpy as np
import torch

class CustomWordEmbeddings(TokenEmbeddings):
    embeddings_name = "custom_word_embeddings"
    def __init__(self, model_path):
        super().__init__()
        self.name = "custom_word_embeddings"
        self.static_embeddings = True
        self.model_path = model_path
        self.model = Word2Vec.load(model_path)

    @property
    def embedding_length(self):
        return self.model.vector_size

    def embed(self, sentences: List[Sentence]) -> List[Sentence]:
        for sentence in sentences:
            for token in sentence.tokens:
                if token.text in self.model.wv:
                    embedding = np.array(self.model.wv[token.text])
                    embedding_tensor = torch.tensor(embedding)  # Convert to a PyTorch tensor
                    token.set_embedding(self.name, embedding_tensor)
                else:
                    # Handle out-of-vocabulary words
                    token.set_embedding(self.name, torch.zeros(self.model.vector_size))
        return sentences

    @classmethod
    def from_params(cls, params: Dict[str, Any]) -> "CustomWordEmbeddings":
        model_path = params.get("model_path")
        return cls(model_path)

    def to_params(self) -> Dict[str, Any]:
        return {"model_path": self.model_path}


In [7]:
from flair.embeddings.base import EMBEDDING_CLASSES
model_path = "word2vec.model"
# Create a custom TokenEmbeddings object
custom_word_embeddings = CustomWordEmbeddings(model_path)

EMBEDDING_CLASSES.update({
    "custom_word_embeddings": custom_word_embeddings
})

# Sequence Tagging

In [8]:
label_type = 'pos'

# Create a label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)

# Create model
model = SequenceTagger(hidden_size=256,
                      embeddings=custom_word_embeddings,
                      tag_dictionary=label_dict,
                      tag_type=label_type)

# Create the trainer and train the model
trainer = ModelTrainer(model, corpus)
trainer.train('pos_tagger_w2v', learning_rate=0.1, mini_batch_size=32, max_epochs=10)

2023-11-16 09:20:11,879 Computing label dictionary. Progress:


0it [00:00, ?it/s]
34989it [00:01, 28574.53it/s]

2023-11-16 09:20:13,165 Dictionary created for label 'pos' with 16 values: part (seen 108378 times), n (seen 85028 times), ppm (seen 70057 times), v (seen 62463 times), punc (seen 43823 times), pron (seen 16539 times), conj (seen 14497 times), adj (seen 12821 times), adv (seen 8663 times), num (seen 4867 times), tn (seen 4734 times), fw (seen 2590 times), int (seen 545 times), abb (seen 283 times), sb (seen 224 times), O (seen 1 times)
2023-11-16 09:20:13,169 SequenceTagger predicts: Dictionary with 16 tags: part, n, ppm, v, punc, pron, conj, adj, adv, num, tn, fw, int, abb, sb, O


2023-11-16 09:20:26,519 ----------------------------------------------------------------------------------------------------
2023-11-16 09:20:26,521 Model: "SequenceTagger(
  (embeddings): CustomWordEmbeddings()
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=18, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2023-11-16 09:20:26,525 ----------------------------------------------------------------------------------------------------
2023-11-16 09:20:26,527 Corpus: 34989 train + 3888 dev + 4320 test sentences
2023-11-16 09:20:26,532 ----------------------------------------------------------------------------------------------------
2023-11-16 09:20:26,532 Train:  34989 sentences
2023-11-16 09:20:26,536         (train_with_dev=False, train_with_test=False)
2023-11-

100%|██████████| 61/61 [00:10<00:00,  6.00it/s]


...

100%|██████████| 61/61 [00:16<00:00,  3.68it/s]


2023-11-16 09:37:14,888 DEV : loss 0.19604137539863586 - f1-score (micro avg)  0.9418
2023-11-16 09:37:15,046  - 0 epochs without improvement
2023-11-16 09:37:15,048 saving best model
2023-11-16 09:37:15,064 ----------------------------------------------------------------------------------------------------
2023-11-16 09:37:15,066 Loading model from best epoch ...
2023-11-16 09:37:15,421 SequenceTagger predicts: Dictionary with 18 tags: part, n, ppm, v, punc, pron, conj, adj, adv, num, tn, fw, int, abb, sb, O, <START>, <STOP>


100%|██████████| 68/68 [00:12<00:00,  5.66it/s]


2023-11-16 09:37:27,968 
Results:
- F-score (micro) 0.9381
- F-score (macro) 0.8474
- Accuracy 0.9381

By class:
              precision    recall  f1-score   support

        part     0.9383    0.9477    0.9430     13207
           n     0.9157    0.9196    0.9176     10494
         ppm     0.9630    0.9875    0.9751      8638
           v     0.9364    0.9398    0.9381      7588
        punc     0.9958    0.9959    0.9958      5421
        pron     0.9600    0.9711    0.9655      2004
        conj     0.8836    0.9196    0.9012      1741
         adj     0.8172    0.7541    0.7843      1541
         adv     0.8883    0.7629    0.8209      1063
         num     0.9444    0.9413    0.9429       596
          tn     0.9463    0.9511    0.9487       593
          fw     0.8389    0.3463    0.4902       361
         int     0.8000    0.6471    0.7154        68
         abb     1.0000    0.5000    0.6667        38
          sb     0.8571    0.6000    0.7059        20

    accuracy         

{'test_score': 0.9381335131995578}

# Predict

In [9]:
# load the model you trained
model = SequenceTagger.load('pos_tagger_w2v/final-model.pt')

# create example sentence
sentence = Sentence('ကျွန်တော့် အတွက် သိပ် အဆင်မပြေ လှ ပါ ဘူး ။')

# predict tags and print
model.predict(sentence)

print(sentence.to_tagged_string())

2023-11-16 09:37:28,321 SequenceTagger predicts: Dictionary with 18 tags: part, n, ppm, v, punc, pron, conj, adj, adv, num, tn, fw, int, abb, sb, O, <START>, <STOP>
Sentence[8]: "ကျွန်တော့် အတွက် သိပ် အဆင်မပြေ လှ ပါ ဘူး ။" → ["ကျွန်တော့်"/pron, "အတွက်"/ppm, "သိပ်"/adv, "အဆင်မပြေ"/v, "လှ"/part, "ပါ"/part, "ဘူး"/part, "။"/punc]


In [10]:
!mv pos_tagger_w2v /content/drive/MyDrive/